In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [2]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [3]:
def load_pdf(data):
    loader=DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [ ]:
extracted_data=load_pdf("data/")

In [ ]:
#extracted_data

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=80)

In [ ]:
text_chunks=text_splitter.split_documents(extracted_data)

In [ ]:
len(text_chunks)

In [4]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\gyanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
C:\Users\gyanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
C:\Users\gyanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1

In [5]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [39]:
query_result=embeddings.embed_query("Hello World")

In [40]:
len(query_result)

384

In [6]:
persist_directory='db'

In [42]:
vectordb=Chroma.from_documents(documents=text_chunks,embedding=embeddings,persist_directory=persist_directory)

In [43]:
vectordb.persist()

C:\Users\gyanb\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [7]:
vectordb=None

In [8]:
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embeddings)

In [9]:
vectordb

In [10]:
retriever=vectordb.as_retriever()

In [11]:
retriever=vectordb.as_retriever(search_kwargs={"k":2})

In [12]:
docs=retriever.invoke("What are allergies")

In [13]:
docs

[Document(page_content='Allergic React ions \nLisa R. Palivos, MD \nKey Points \n• Epinephrine is the first-line medication for the treat\xad\nment of anaphylaxis. Second-line treatment consists of \nantihistamines and corticosteroids. \n• Urticaria may be the first sign of what might progress to \nangioedema or anaphylaxis. \nINTRODUCTION \nAn allergic reaction is the body\'s way of responding to \nforeign substances that come in contact with the skin, nose, \neyes, respiratory tract or gastrointestinal tract. Examples of \nallergens are dust, pollen, plants, medications, foods, latex, \nand insect bites. Anything can be an allergen. Allergic \nreactions can range from mild local urticarial eruptions \nto severe and life-threatening airway obstruction, respira \xad\ntory failure, and circulatory colla pse. Urticaria, or "hives;\' \nis an immunoglobulin E (IgE)-m ediated hypersensitivity \nreaction to an allergen resulting in red, raised wheals that \nitch and sting. Circulating antibo

In [11]:
prompt_template='''
Use the following pieces of information to answer the user's question.
If you don't know the answer,just say that you don't know,don't try to makeup the answer.
Context: {context}
Question: {question}

Only return the helpful aswer below and nothing else.
Helpful Answer:
'''

In [12]:
PROMPT=PromptTemplate(template=prompt_template,input_variales=["context","question"])
chain_type_kwargs={"prompt":PROMPT}

In [13]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",model_type="llama",config={'max_new_tokens':2048,'context_length' : 1024,'temperature':0.8})

In [14]:
qa=RetrievalQA.from_chain_type(llm=llm,chain_type="stuff",retriever=vectordb.as_retriever(search_kwargs={"k":2}),return_source_documents=True,chain_type_kwargs=chain_type_kwargs)

In [15]:
import warnings 
warnings.filterwarnings('ignore') 

In [16]:
 user_input=input(f"Input Prompt:")
result=qa({"query":user_input})
print("Response: ",result["result"])

Input Prompt: what ia achne


Response:  An answer to this question is not possible without more context or information about the specific person being referred to. Personality disorders are complex mental health conditions that can manifest in a variety of ways, and it would be impossible to accurately identify the type of personality disorder someone has based solely on the information provided.

To provide a proper diagnosis and treatment plan, a qualified mental health professional would need to conduct a comprehensive evaluation of the individual's symptoms, behaviors, and medical history. This would involve several sessions of psychotherapy and may also include psychological testing and assessment tools.

In summary, while you may have some general knowledge about personality disorders, it is not possible to accurately identify the specific type of disorder someone has without a thorough evaluation by a qualified mental health professional.


In [17]:
while True:
    user_input=input(f"Input Prompt:")
    if user_input=="Exit":
        break
    result=qa({"query":user_input})
    print("Response: ",result["result"])

Input Prompt: what is allergies


Response:  Allergy is an abnormal immune response to a substance that is typically harmless but can cause an exaggerated reaction in some people. Allergic reactions occur when the body's immune system mistakenly identifies a harmless substance as a threat and attempts to fight it off, leading to inflammation and damage. Common allergy triggers include pollen, dust mites, mold, pet dander, and certain foods like nuts or shellfish. Symptoms of an allergic reaction can range from mild discomfort to life-threatening reactions, depending on the severity of the reaction. Treatment options include medication, immunotherapy, and avoidance of the allergen.


Input Prompt: what is malaria


Response:  Malaria is a serious disease caused by a parasitic infection, typically transmitted through the bite of an infected mosquito. The parasites that cause malaria are known as Plasmodium species, and there are several different species that can cause the disease, including P. falciparum, P. vivax, and P. ovale. Malaria can cause a wide range of symptoms, including fever, chills, headache, muscle and joint pain, fatigue, and in severe cases, it can lead to organ failure and death. Diagnosis of malaria is typically made through a combination of medical history, physical examination, and laboratory tests, such as microscopy or rapid diagnostic tests (RDTs) that detect the presence of the parasite in the blood. Treatment for malaria usually involves antimalarial drugs, which can help to eliminate the parasite from the body and alleviate symptoms. Prevention measures for malaria include using insecticide-treated bed nets, wearing protective clothing, and using anti-mosquito agents wh

Input Prompt: Exit
